<a href="https://colab.research.google.com/github/zjc664656505/BQ_PROJECT/blob/main/BQ_LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load data from Google Drive

In [111]:
#Mount to your google drive
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [112]:
#define method for feature selection
def feature_selection(dataframe, feature_list):
  '''
    This is a method which returns the new dataframe in which the features are selected
  '''
  return dataframe[feature_list]

In [113]:
data = pd.read_csv("/content/drive/MyDrive/BrightQuery_Project/meta-model/new_merged_meta_data.csv")

In [114]:
primary_key = ['bq_id', 'bq_year']
other_group_features = ['bq_sector_name', 'bq_business_code', 'bq_company_address1_state']

In [116]:
# get the company info
categorical_variable = feature_selection(data, ['bq_id']+other_group_key)
company_info = categorical_variable.drop_duplicates(subset=['bq_id'])
# Convert categorical variable into dummy/indicator variables.
company_info = pd.get_dummies(data=company_info)

In [118]:
company_info

,bq_id,bq_business_code,bq_sector_name_Accommodation and Food Services,bq_sector_name_Administrative and Support and Waste Management and Remediation Services,"bq_sector_name_Agriculture, Forestry, Fishing and Hunting","bq_sector_name_Arts, Entertainment, and Recreation",bq_sector_name_Construction,bq_sector_name_Educational Services,bq_sector_name_Finance and Insurance,bq_sector_name_Health Care and Social Assistance,bq_sector_name_Information,bq_sector_name_Management of Companies (Holding Companies),bq_sector_name_Manufacturing,bq_sector_name_Mining,bq_sector_name_Other Services,"bq_sector_name_Professional, Scientific, and Technical Services",bq_sector_name_Real Estate and Rental and Leasing,bq_sector_name_Retail Trade,bq_sector_name_Transportation and Warehousing,bq_sector_name_Utilities,bq_sector_name_Wholesale Trade,bq_company_address1_state_AK,bq_company_address1_state_AL,bq_company_address1_state_AR,bq_company_address1_state_AZ,bq_company_address1_state_CA,bq_company_address1_state_CO,bq_company_address1_state_CT,bq_company_address1_state_DC,bq_company_address1_state_DE,bq_company_address1_state_FL,bq_company_address1_state_GA,bq_company_address1_state_HI,bq_company_address1_state_IA,bq_company_address1_state_ID,bq_company_address1_state_IL,bq_company_address1_state_IN,bq_company_address1_state_KS,bq_company_address1_state_KY,bq_company_address1_state_LA,bq_company_address1_state_MA,bq_company_address1_state_MD,bq_company_address1_state_ME,bq_company_address1_state_MI,bq_company_address1_state_MN,bq_company_address1_state_MO,bq_company_address1_state_MS,bq_company_address1_state_MT,bq_company_address1_state_NC,bq_company_address1_state_ND,bq_company_address1_state_NE,bq_company_address1_state_NH,bq_company_address1_state_NJ,bq_company_address1_state_NM,bq_company_address1_state_NV,bq_company_address1_state_NY,bq_company_address1_state_OH,bq_company_address1_state_OK,bq_company_address1_state_OR,bq_company_address1_state_PA,bq_company_address1_state_PR,bq_company_address1_state_RI,bq_company_address1_state_SC,bq_company_address1_state_SD,bq_company_address1_state_TN,bq_company_address1_state_TX,bq_company_address1_state_UT,bq_company_address1_state_VA,bq_company_address1_state_VI,bq_company_address1_state_VT,bq_company_address1_state_WA,bq_company_address1_state_WI,bq_company_address1_state_WV
0,412416919,221210,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,136315656,522110,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,666958061,722511,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,105749915,522110,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,218284269,522110,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16057,484886126,334200,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16421,500295400,333100,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17291,732838577,541700,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
18385,230033736,561450,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [124]:
# define method for Relative Absolute Error (RAE)
def RAE(y_true, y_pred):
  true = np.array(y_true)
  pred = np.array(y_pred)
  numerator =  np.sum(np.abs(pred - true))
  denominator = np.sum(np.abs(true - np.mean(true)))
  return numerator/denominator

# define method for Mean Absolute Error (MAE)
def MAE(y_true, y_pred):
  return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))

## Net Income

### Transfer data with years as variables

In [120]:
NetIncomeFeatures = pd.read_csv("/content/drive/MyDrive/BrightQuery_Project/SelectedFeatures/NetIncomeFeatures.csv")
NetIncomeFeatures = NetIncomeFeatures['Net Income Features'].tolist()

# get the data
net_incoome_public = feature_selection(data, primary_key+['bq_net_income']+NetIncomeFeatures)


In [121]:
# Tranfer the long date to wide data
new_pd = net_incoome_public[(net_incoome_public.bq_year == 2015)].drop(columns=['bq_year'])
new_pd.columns = new_pd.columns+"_"+str(2015)
new_pd = new_pd.rename(columns={'bq_id_'+str(2015): 'bq_id'})

for i in range(2016, 2021):
  next_year_net_income = net_incoome_public[(net_incoome_public.bq_year == i)].drop(columns=['bq_year'])
  next_year_net_income.columns = next_year_net_income.columns+"_"+str(i)
  next_year_net_income = next_year_net_income.rename(columns={'bq_id_'+str(i): 'bq_id'})
  new_pd = new_pd.merge(next_year_net_income, how="outer", on='bq_id')

new_pd

,bq_id,bq_net_income_2015,bq_prepaid_legal_ind_2015,bq_severance_pay_ind_2015,bq_stock_bonus_ind_2015,bq_terminate_any_ind_2015,bq_final_filing_any_ind_2015,bq_fail_trans_contrib_pens_ind_2015,bq_amended_ind_2015,bq_final_filing_pens_ind_2015,bq_fail_trans_contrib_any_ind_2015,bq_vision_insurance_ind_2015,bq_temp_disability_insurance_ind_2015,bq_death_benefits_ind_2015,bq_defined_benefit_pens_ind_2015,bq_money_purchase_plan_ind_2015,bq_fail_provide_benf_due_pens_ind_2015,bq_no_co_contribs_ind_2015,bq_terminate_pens_ind_2015,bq_fail_provide_benf_due_any_ind_2015,bq_supp_unemploy_ins_ind_2015,bq_fail_provide_benf_due_welf_ind_2015,bq_net_income_2016,bq_prepaid_legal_ind_2016,bq_severance_pay_ind_2016,bq_stock_bonus_ind_2016,bq_terminate_any_ind_2016,bq_final_filing_any_ind_2016,bq_fail_trans_contrib_pens_ind_2016,bq_amended_ind_2016,bq_final_filing_pens_ind_2016,bq_fail_trans_contrib_any_ind_2016,bq_vision_insurance_ind_2016,bq_temp_disability_insurance_ind_2016,bq_death_benefits_ind_2016,bq_defined_benefit_pens_ind_2016,bq_money_purchase_plan_ind_2016,bq_fail_provide_benf_due_pens_ind_2016,bq_no_co_contribs_ind_2016,bq_terminate_pens_ind_2016,...,bq_severance_pay_ind_2019,bq_stock_bonus_ind_2019,bq_terminate_any_ind_2019,bq_final_filing_any_ind_2019,bq_fail_trans_contrib_pens_ind_2019,bq_amended_ind_2019,bq_final_filing_pens_ind_2019,bq_fail_trans_contrib_any_ind_2019,bq_vision_insurance_ind_2019,bq_temp_disability_insurance_ind_2019,bq_death_benefits_ind_2019,bq_defined_benefit_pens_ind_2019,bq_money_purchase_plan_ind_2019,bq_fail_provide_benf_due_pens_ind_2019,bq_no_co_contribs_ind_2019,bq_terminate_pens_ind_2019,bq_fail_provide_benf_due_any_ind_2019,bq_supp_unemploy_ins_ind_2019,bq_fail_provide_benf_due_welf_ind_2019,bq_net_income_2020,bq_prepaid_legal_ind_2020,bq_severance_pay_ind_2020,bq_stock_bonus_ind_2020,bq_terminate_any_ind_2020,bq_final_filing_any_ind_2020,bq_fail_trans_contrib_pens_ind_2020,bq_amended_ind_2020,bq_final_filing_pens_ind_2020,bq_fail_trans_contrib_any_ind_2020,bq_vision_insurance_ind_2020,bq_temp_disability_insurance_ind_2020,bq_death_benefits_ind_2020,bq_defined_benefit_pens_ind_2020,bq_money_purchase_plan_ind_2020,bq_fail_provide_benf_due_pens_ind_2020,bq_no_co_contribs_ind_2020,bq_terminate_pens_ind_2020,bq_fail_provide_benf_due_any_ind_2020,bq_supp_unemploy_ins_ind_2020,bq_fail_provide_benf_due_welf_ind_2020
0,412416919,41140000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44675000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71711000.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,136315656,92258000.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,113747000.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126205000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,666958061,5390000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4030000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-4688000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,105749915,6343000.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7331000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14072000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,218284269,5716767.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5594011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [122]:
df = new_pd.dropna()
drop_columns = (net_incoome_public.columns[2:] + '_'+str(2020)).tolist()
X = df.drop(columns=drop_columns)
y = df['bq_net_income_2020']
# merge X with categorical data
X = X.merge(company_info, how="inner", on='bq_id')

In [126]:
X['bq_id'] = X['bq_id'].astype("category")
X['bq_business_code'] = X['bq_business_code'].astype("category")

### Model training and results

In [128]:
# Split the data into training/testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)
r_sq = model.score(X_test, y_test)
print('Coefficient of Determination (R Squared):', r_sq)

from sklearn.metrics import mean_squared_error
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
RAE_train_error = RAE(y_train, y_train_pred)
RAE_test_error = RAE(y_test, y_test_pred)
print('Relative Absolute Error (RAE)_training:', RAE_train_error)
print('Relative Absolute Error (RAE)_test', RAE_test_error)

Coefficient of Determination (R Squared): 0.7990053509909367
Relative Absolute Error (RAE)_training: 0.6268781997603851
Relative Absolute Error (RAE)_test 0.5873289148643754


## Revenue

### Transfer data with years as variables

In [129]:
RevenueFeatures = pd.read_csv("/content/drive/MyDrive/BrightQuery_Project/SelectedFeatures/RevenueFeatures.csv")
RevenueFeatures = RevenueFeatures['Revenue Features'].tolist()

# get the data
revenue_public = feature_selection(data, primary_key+['bq_revenue']+RevenueFeatures)

In [130]:
# Tranfer the long date to wide data
new_pd = revenue_public[(revenue_public.bq_year == 2015)].drop(columns=['bq_year'])
new_pd.columns = new_pd.columns+"_"+str(2015)
new_pd = new_pd.rename(columns={'bq_id_'+str(2015): 'bq_id'})

for i in range(2016, 2021):
  next_year_revenue = revenue_public[(revenue_public.bq_year == i)].drop(columns=['bq_year'])
  next_year_revenue.columns = next_year_revenue.columns+"_"+str(i)
  next_year_revenue = next_year_revenue.rename(columns={'bq_id_'+str(i): 'bq_id'})
  new_pd = new_pd.merge(next_year_revenue, how="outer", on='bq_id')

new_pd

,bq_id,bq_revenue_2015,bq_final_filing_any_ind_2015,bq_supp_unemploy_ins_ind_2015,bq_prepaid_legal_ind_2015,bq_severance_pay_ind_2015,bq_defined_benefit_pens_ind_2015,bq_terminate_any_ind_2015,bq_fail_trans_contrib_pens_ind_2015,bq_amended_ind_2015,bq_final_filing_pens_ind_2015,bq_fail_trans_contrib_any_ind_2015,bq_vision_insurance_ind_2015,bq_temp_disability_insurance_ind_2015,bq_death_benefits_ind_2015,bq_stock_bonus_ind_2015,bq_fail_provide_benf_due_pens_ind_2015,bq_terminate_pens_ind_2015,bq_fail_provide_benf_due_any_ind_2015,bq_dental_insurance_ind_2015,bq_code_section_401k_ind_2015,bq_money_purchase_plan_ind_2015,bq_revenue_2016,bq_final_filing_any_ind_2016,bq_supp_unemploy_ins_ind_2016,bq_prepaid_legal_ind_2016,bq_severance_pay_ind_2016,bq_defined_benefit_pens_ind_2016,bq_terminate_any_ind_2016,bq_fail_trans_contrib_pens_ind_2016,bq_amended_ind_2016,bq_final_filing_pens_ind_2016,bq_fail_trans_contrib_any_ind_2016,bq_vision_insurance_ind_2016,bq_temp_disability_insurance_ind_2016,bq_death_benefits_ind_2016,bq_stock_bonus_ind_2016,bq_fail_provide_benf_due_pens_ind_2016,bq_terminate_pens_ind_2016,bq_fail_provide_benf_due_any_ind_2016,...,bq_supp_unemploy_ins_ind_2019,bq_prepaid_legal_ind_2019,bq_severance_pay_ind_2019,bq_defined_benefit_pens_ind_2019,bq_terminate_any_ind_2019,bq_fail_trans_contrib_pens_ind_2019,bq_amended_ind_2019,bq_final_filing_pens_ind_2019,bq_fail_trans_contrib_any_ind_2019,bq_vision_insurance_ind_2019,bq_temp_disability_insurance_ind_2019,bq_death_benefits_ind_2019,bq_stock_bonus_ind_2019,bq_fail_provide_benf_due_pens_ind_2019,bq_terminate_pens_ind_2019,bq_fail_provide_benf_due_any_ind_2019,bq_dental_insurance_ind_2019,bq_code_section_401k_ind_2019,bq_money_purchase_plan_ind_2019,bq_revenue_2020,bq_final_filing_any_ind_2020,bq_supp_unemploy_ins_ind_2020,bq_prepaid_legal_ind_2020,bq_severance_pay_ind_2020,bq_defined_benefit_pens_ind_2020,bq_terminate_any_ind_2020,bq_fail_trans_contrib_pens_ind_2020,bq_amended_ind_2020,bq_final_filing_pens_ind_2020,bq_fail_trans_contrib_any_ind_2020,bq_vision_insurance_ind_2020,bq_temp_disability_insurance_ind_2020,bq_death_benefits_ind_2020,bq_stock_bonus_ind_2020,bq_fail_provide_benf_due_pens_ind_2020,bq_terminate_pens_ind_2020,bq_fail_provide_benf_due_any_ind_2020,bq_dental_insurance_ind_2020,bq_code_section_401k_ind_2020,bq_money_purchase_plan_ind_2020
0,412416919,459244000.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,498860000.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,483154000.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,136315656,308733000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,406174000.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,446746000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,666958061,145863000.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,150582000.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,106490000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,105749915,38394000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,41342000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,56012000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,218284269,24787824.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,24707521.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.

In [131]:
df = new_pd.dropna()
drop_columns = (revenue_public.columns[2:] + '_'+str(2020)).tolist()
X = df.drop(columns=drop_columns)
y = df['bq_revenue_2020']
# merge X with categorical data
X = X.merge(company_info, how="inner", on='bq_id')

X['bq_id'] = X['bq_id'].astype("category")
X['bq_business_code'] = X['bq_business_code'].astype("category")

### Model training and results

In [138]:
# Split the data into training/testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)
r_sq = model.score(X_test, y_test)
print('Coefficient of Determination (R Squared):', r_sq)

from sklearn.metrics import mean_squared_error
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
RAE_train_error = RAE(y_train, y_train_pred)
RAE_test_error = RAE(y_test, y_test_pred)
print('Relative Absolute Error (RAE)_training:', RAE_train_error)
print('Relative Absolute Error (RAE)_test', RAE_test_error)

Coefficient of Determination (R Squared): 0.9786105460540238
Relative Absolute Error (RAE)_training: 0.13368219887892838
Relative Absolute Error (RAE)_test 0.1663789890404811


## Ebitda

### Transfer data with years as variables

In [139]:
EbitdaFeatures = pd.read_csv("/content/drive/MyDrive/BrightQuery_Project/SelectedFeatures/EbitdaFeatures.csv")
EbitdaFeatures = EbitdaFeatures['Ebitda Features'].tolist()

# get the data
ebitda_public = feature_selection(data, primary_key+['bq_ebitda']+EbitdaFeatures)

In [140]:
# Tranfer the long date to wide data
new_pd = ebitda_public[(ebitda_public.bq_year == 2015)].drop(columns=['bq_year'])
new_pd.columns = new_pd.columns+"_"+str(2015)
new_pd = new_pd.rename(columns={'bq_id_'+str(2015): 'bq_id'})

for i in range(2016, 2021):
  next_year_ebitda = ebitda_public[(ebitda_public.bq_year == i)].drop(columns=['bq_year'])
  next_year_ebitda.columns = next_year_ebitda.columns+"_"+str(i)
  next_year_ebitda = next_year_ebitda.rename(columns={'bq_id_'+str(i): 'bq_id'})
  new_pd = new_pd.merge(next_year_ebitda, how="outer", on='bq_id')

new_pd

,bq_id,bq_ebitda_2015,bq_terminate_any_ind_2015,bq_prepaid_legal_ind_2015,bq_severance_pay_ind_2015,bq_defined_benefit_pens_ind_2015,bq_stock_bonus_ind_2015,bq_final_filing_any_ind_2015,bq_fail_trans_contrib_pens_ind_2015,bq_terminate_pens_ind_2015,bq_final_filing_pens_ind_2015,bq_fail_trans_contrib_any_ind_2015,bq_fail_provide_benf_due_any_ind_2015,bq_vision_insurance_ind_2015,bq_temp_disability_insurance_ind_2015,bq_supp_unemploy_ins_ind_2015,bq_death_benefits_ind_2015,bq_money_purchase_plan_ind_2015,bq_amended_ind_2015,bq_health_insurance_ind_2015,bq_profit_sharing_ind_2015,bq_fail_trans_contrib_welf_ind_2015,bq_ebitda_2016,bq_terminate_any_ind_2016,bq_prepaid_legal_ind_2016,bq_severance_pay_ind_2016,bq_defined_benefit_pens_ind_2016,bq_stock_bonus_ind_2016,bq_final_filing_any_ind_2016,bq_fail_trans_contrib_pens_ind_2016,bq_terminate_pens_ind_2016,bq_final_filing_pens_ind_2016,bq_fail_trans_contrib_any_ind_2016,bq_fail_provide_benf_due_any_ind_2016,bq_vision_insurance_ind_2016,bq_temp_disability_insurance_ind_2016,bq_supp_unemploy_ins_ind_2016,bq_death_benefits_ind_2016,bq_money_purchase_plan_ind_2016,bq_amended_ind_2016,...,bq_prepaid_legal_ind_2019,bq_severance_pay_ind_2019,bq_defined_benefit_pens_ind_2019,bq_stock_bonus_ind_2019,bq_final_filing_any_ind_2019,bq_fail_trans_contrib_pens_ind_2019,bq_terminate_pens_ind_2019,bq_final_filing_pens_ind_2019,bq_fail_trans_contrib_any_ind_2019,bq_fail_provide_benf_due_any_ind_2019,bq_vision_insurance_ind_2019,bq_temp_disability_insurance_ind_2019,bq_supp_unemploy_ins_ind_2019,bq_death_benefits_ind_2019,bq_money_purchase_plan_ind_2019,bq_amended_ind_2019,bq_health_insurance_ind_2019,bq_profit_sharing_ind_2019,bq_fail_trans_contrib_welf_ind_2019,bq_ebitda_2020,bq_terminate_any_ind_2020,bq_prepaid_legal_ind_2020,bq_severance_pay_ind_2020,bq_defined_benefit_pens_ind_2020,bq_stock_bonus_ind_2020,bq_final_filing_any_ind_2020,bq_fail_trans_contrib_pens_ind_2020,bq_terminate_pens_ind_2020,bq_final_filing_pens_ind_2020,bq_fail_trans_contrib_any_ind_2020,bq_fail_provide_benf_due_any_ind_2020,bq_vision_insurance_ind_2020,bq_temp_disability_insurance_ind_2020,bq_supp_unemploy_ins_ind_2020,bq_death_benefits_ind_2020,bq_money_purchase_plan_ind_2020,bq_amended_ind_2020,bq_health_insurance_ind_2020,bq_profit_sharing_ind_2020,bq_fail_trans_contrib_welf_ind_2020
0,412416919,115001000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,123148000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,178694000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
1,136315656,150819000.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,191199000.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,163741000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,666958061,12639000.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,11097000.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,-3012000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,105749915,11415000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,13291000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,20592000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,218284269,9096537.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,8451996.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,

In [142]:
df = new_pd.dropna()
drop_columns = (ebitda_public.columns[2:] + '_'+str(2020)).tolist()
X = df.drop(columns=drop_columns)
y = df['bq_ebitda_2020']
# merge X with categorical data
X = X.merge(company_info, how="inner", on='bq_id')

X['bq_id'] = X['bq_id'].astype("category")
X['bq_business_code'] = X['bq_business_code'].astype("category")

### Model training and results

In [143]:
# Split the data into training/testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)
r_sq = model.score(X_test, y_test)
print('Coefficient of Determination (R Squared):', r_sq)

from sklearn.metrics import mean_squared_error
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
RAE_train_error = RAE(y_train, y_train_pred)
RAE_test_error = RAE(y_test, y_test_pred)
print('Relative Absolute Error (RAE)_training:', RAE_train_error)
print('Relative Absolute Error (RAE)_test', RAE_test_error)

Coefficient of Determination (R Squared): 0.8929926795493288
Relative Absolute Error (RAE)_training: 0.3933896323119358
Relative Absolute Error (RAE)_test 0.38453264619334737
